In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

from IPython.display import display  # noqa: F401

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Kinematics

In [ ]:
import sympy as sp
from IPython.display import Math

from ampform.io import aslatex

:::{margin}
This notebook originates from {doc}`compwa-org:report/017`.
:::

Kinematics for a three-body decay $0 \to 123$ can be fully described by two **Mandelstam variables** $\sigma_1, \sigma_2$, because the third variable $\sigma_3$ can be expressed in terms $\sigma_1, \sigma_2$, the mass $m_0$ of the initial state, and the masses $m_1, m_2, m_3$ of the final state. As can be seen, the roles of $\sigma_1, \sigma_2, \sigma_3$ are interchangeable.

```{margin}
See Eq. (1.2) in {cite}`bycklingParticleKinematics1973`
```

In [ ]:
from ampform.kinematics.phasespace import compute_third_mandelstam

m0, m1, m2, m3 = sp.symbols("m:4")
s1, s2, s3 = sp.symbols("sigma1:4")
s3_expr = compute_third_mandelstam(s1, s2, m0, m1, m2, m3)

latex = aslatex({s3: s3_expr})
Math(latex)

The phase space is defined by the closed area that satisfies the condition $\phi(\sigma_1,\sigma_2) \leq 0$, where $\phi$ is a **Kibble function**:


```{margin}
See §V.2 in {cite}`bycklingParticleKinematics1973`
```

In [ ]:
from ampform.kinematics.phasespace import Kibble

kibble = Kibble(s1, s2, s3, m0, m1, m2, m3)

latex = aslatex({kibble: kibble.evaluate()})
Math(latex)

and $\lambda$ is the **Källén function**:

In [ ]:
from ampform.kinematics.phasespace import Kallen

x, y, z = sp.symbols("x:z")
kallen = Kallen(x, y, z)

latex = aslatex({kallen: kallen.evaluate()})
Math(latex)

Any distribution over the phase space can now be defined using a two-dimensional grid over a Mandelstam pair $\sigma_1,\sigma_2$ of choice, with the condition $\phi(\sigma_1,\sigma_2)<0$ selecting the values that are physically allowed.

In [ ]:
from ampform.kinematics.phasespace import is_within_phasespace

is_within_phasespace(s1, s2, m0, m1, m2, m3)

See {doc}`compwa-org:report/017` for an interactive visualization of the phase space region and an analytic expression for the phase space boundary.